In [2]:
import json
import time
import requests
import sqlite3
import pandas as pd

##### Capitalisation boursière = Cours actuel x Offre en circulation

##### Désigne la valeur marchande totale de l’offre en circulation d’une crypto-monnaie. Semblable à la mesure du marché boursier  qui multiplie le cours par action par les actions facilement disponibles sur le marché (non détenues ni bloquées par des  initiés, des gouvernements).

In [3]:
## REQUEST
#Obtenir le prix actuel de toutes les crypto-monnaies dans toutes les autres devises prises en charge dont vous avez besoin.
response_5 = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=bitcoin%2Clitecoin&vs_currencies=eur%2Cusd&include_market_cap=true&include_24hr_vol=true&include_24hr_change=true&include_last_updated_at=true&precision=2")
simple_price = response_5.json()
#print(simple_price)
#print(simple_price["bitcoin"]["eur_market_cap"])


In [6]:
## PARSE
for dict in simple_price["bitcoin"]:
    eur = simple_price["bitcoin"]["eur"]
    eur_market_cap = simple_price["bitcoin"]["eur_market_cap"]
    eur_24h_vol = simple_price["bitcoin"]["eur_24h_vol"]
    eur_24h_change = simple_price["bitcoin"]["eur_24h_change"]
    usd = simple_price["bitcoin"]["usd"]
    usd_market_cap = simple_price["bitcoin"]["usd_market_cap"]
    usd_24h_vol = simple_price["bitcoin"]["usd_24h_vol"]
    usd_24h_change = simple_price["bitcoin"]["usd_24h_change"]
    last_updated_at = simple_price["bitcoin"]["last_updated_at"]
print('euro:',eur,"\n"'dollar:',usd,"\n"'capitalisation boursiere €:',eur_market_cap,"\n"'Volume de négociation sur 24 h:',eur_24h_vol,"\n"'derniere Maj Prix:',last_updated_at)
print(type(last_updated_at))

euro: 24114.42 
dollar: 26031.48 
capitalisation boursiere €: 469444244554.7783 
Volume de négociation sur 24 h: 488051495.8954078 
derniere Maj Prix: 1693051798
<class 'int'>


In [4]:
## BDD
#création de la Bdd SQLite si existe + création de la connexion
conn = sqlite3.connect("ProjectBTC.db")
cur = conn.cursor()

In [ ]:
## Bdd ajout colonne 
#req_1 = """ALTER TABLE btc ADD COLUMN eur_market_cap INTEGER"""
req_1 = """ALTER TABLE btc ADD COLUMN eur_24h_vol INTEGER"""
# Exécution de la requete 
cur.execute(req_1) 
# Envoyer la requete 
conn.commit() 
# Fermer la connexion 
conn.close()

In [ ]:
## BDD
# requete création table si n'existe pas
req = """CREATE TABLE IF NOT EXISTS btc(
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    eur INTEGER NOT NULL, 
    usd INTEGER NOT NULL,    
    last_update INTEGER NOT NULL,
    eur_market_cap INTEGER NOT NULL,
    eur_24h_vol INTEGER NOT NULL)""" 
# Exécution de la requete 
cur.execute(req) 
# Envoyer la requete 
conn.commit() 
# Fermer la connexion 
conn.close()

In [84]:
## Transformation = TIME

#last_updated_1 = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(last_updated_at))
last_updated_1 = time.localtime(last_updated_at)
formatted_local_time = time.strftime("%Y-%m-%d %H:%M:%S", last_updated_1) 
print(last_updated_1)
print(type(last_updated_1))
print(formatted_local_time)



time.struct_time(tm_year=2023, tm_mon=8, tm_mday=23, tm_hour=9, tm_min=54, tm_sec=40, tm_wday=2, tm_yday=235, tm_isdst=1)
<class 'time.struct_time'>
2023-08-23 09:54:40


In [6]:
## INSERT
conn = sqlite3.connect("ProjectBTC.db")
cur = conn.cursor()
req1 = 'INSERT INTO btc(eur,usd,last_update,eur_market_cap,eur_24h_vol) VALUES(?,?,?,?,?)'
val1 = (eur,usd,last_updated_1,eur_market_cap,eur_24h_vol)
cur.execute(req1,val1)
conn.commit() 
cur.close()

In [ ]:
## UPDATE unixepoch line

#conn = sqlite3.connect("ProjectBTC.db")
#cur = connexion.cursor()
#req2 = 'update btc set last_update = datetime(last_update, 'unixepoch', 'localtime') where last_update like '16%';'

#cur.execute(req2)

#cur.close()

In [7]:
## PRINT
cur = conn.cursor()
# cur.execute("select * from btc")
#cur.execute("select eur,usd,last_update,eur_market_cap,eur_24h_vol from btc order by last_update DESC limit(10)")
cur.execute("select * from btc WHERE DATE(last_update) = DATE('now')")
records = cur.fetchall()

print(records[0])

(7292, 23979.29, 26015.63, '2023-08-23 01:14:56', 466677140072.07733, 9329058234.743841)


In [10]:
#DATAFRAME

df_btc = pd.read_sql('''select * from btc WHERE DATE(last_update) = DATE('now')''',conn,)
#df_btc = pd.read_sql('select eur,usd,last_update,eur_market_cap,eur_24h_vol from btc order by last_update',conn,)
#df_btc = pd.read_sql('''select eur,usd,last_update from btc where last_update like '2023-08-17%' ''',conn,)
#df_btc = pd.read_sql('select eur,usd,last_update from btc order by last_update DESC limit(10)',conn,)
df_btc

,id,eur,usd,last_update,eur_market_cap,eur_24h_vol
0,7292,23979.29,26015.63,2023-08-23 01:14:56,4.666771e+11,9.329058e+09
1,7293,23982.77,26021.53,2023-08-23 01:19:55,4.666771e+11,1.321158e+10
2,7294,23953.83,25998.62,2023-08-23 01:24:56,4.665311e+11,1.083450e+10
3,7295,23953.83,25998.62,2023-08-23 01:24:56,4.665311e+11,1.083450e+10
4,7296,23975.64,26022.43,2023-08-23 01:35:02,4.667246e+11,1.047794e+10
...,...,...,...,...,...,...
77,7369,24073.46,26045.52,2023-08-23 07:39:58,4.674576e+11,9.402202e+09
78,7370,24058.96,26022.46,2023-08-23 07:44:55,4.685672e+11,1.259141e+10
79,7371,24058.96,26022.46,2023-08-23 07:44:55,4.685672e+11,1.259141e+10
80,7372,24056.47,26041.49,2023-08-23 07:54:40,4.685842e+11,1.081903e+10


In [19]:
# DETECTION DE DOUBLON
df_btc_doublon = pd.read_sql('''SELECT *, COUNT(*) AS nbr_doublon FROM btc where DATE(last_update) = DATE('now') GROUP BY last_update HAVING COUNT(*) > 1 ''',conn,)
#df_btc_doublon = pd.read_sql('''SELECT COUNT(*) AS nbr_doublon, eur,last_update,eur_market_cap FROM btc  where last_update like '2023-08-05%' GROUP BY last_update HAVING COUNT(*) > 1 ''',conn,)
df_btc_doublon


,id,eur,usd,last_update,eur_market_cap,eur_24h_vol,nbr_doublon
0,7294,23953.83,25998.62,2023-08-23 01:24:56,4.665311e+11,1.083450e+10,2
1,7298,24008.26,26052.56,2023-08-23 01:44:58,4.665688e+11,1.376598e+10,3
2,7303,24002.12,26055.53,2023-08-23 02:09:54,4.668559e+11,6.393497e+09,2
3,7305,23977.55,26028.82,2023-08-23 02:19:56,4.671402e+11,1.272023e+10,2
4,7308,23983.53,26030.52,2023-08-23 02:35:00,4.669033e+11,8.865623e+09,2
5,7311,23980.36,26034.28,2023-08-23 02:49:54,4.669458e+11,1.379545e+10,2
6,7313,23981.85,26029.91,2023-08-23 03:00:02,4.667686e+11,1.081206e+10,2
7,7319,23954.76,25998.59,2023-08-23 03:30:02,4.667386e+11,1.166482e+10,2
8,7324,23936.47,25990.61,2023-08-23 03:55:02,4.659783e+11,1.115889e+10,2
9,7325,23946.98,26000.33,2023-08-23 04:00:01,4.659783e+11,1.383363e+10,2


In [33]:
# NBRE DE DOUBLONS => df_btc
df_btc.duplicated(subset='last_update').sum()

25

In [42]:
# BOOLEEN TRUE = DOUBLON => df_btc
#df_btc.duplicated(subset='last_update')
df_btc.duplicated(subset='last_update').value_counts()

False    57
True     25
Name: count, dtype: int64

In [46]:
# SUPPRESSION DE DOUBLON => df_btc => df_btc_no_duplicated
df_btc_no_duplicated = df_btc.drop_duplicates(subset='last_update')
df_btc_no_duplicated

,id,eur,usd,last_update,eur_market_cap,eur_24h_vol
0,7292,23979.29,26015.63,2023-08-23 01:14:56,4.666771e+11,9.329058e+09
1,7293,23982.77,26021.53,2023-08-23 01:19:55,4.666771e+11,1.321158e+10
2,7294,23953.83,25998.62,2023-08-23 01:24:56,4.665311e+11,1.083450e+10
4,7296,23975.64,26022.43,2023-08-23 01:35:02,4.667246e+11,1.047794e+10
5,7297,23968.61,26011.31,2023-08-23 01:40:00,4.667246e+11,7.798841e+09
6,7298,24008.26,26052.56,2023-08-23 01:44:58,4.665688e+11,1.376598e+10
9,7301,23987.32,26041.25,2023-08-23 02:00:00,4.677180e+11,1.341287e+10
10,7302,24004.93,26056.15,2023-08-23 02:04:55,4.668559e+11,1.372518e+10
11,7303,24002.12,26055.53,2023-08-23 02:09:54,4.668559e+11,6.393497e+09
13,7305,23977.55,26028.82,2023-08-23 02:19:56,4.671402e+11,1.272023e+10


In [47]:
# NBRE DE DOUBLONS => df_btc_no_duplicated
df_btc_no_duplicated.duplicated(subset='last_update').sum()

0

In [50]:
# BOOLEEN TRUE = DOUBLON => df_btc_no_duplicated
df_btc_no_duplicated.duplicated(subset='last_update').value_counts()

False    57
Name: count, dtype: int64

In [ ]:
# SUPPRESSION COLONNE 
#df_btc_0 = df_btc.drop(['last_update'],axis=1)
#df_btc_0

In [ ]:
## TRANSFORMATION DATAFRAME
#df_btc_1 = pd.to_datetime(df_btc['last_update'], format='ISO8601')
#df_btc_1

In [ ]:
## TRANSFORMATION DATAFRAME
# Convert datetype to string

#df_btc_1 = df_btc['last_update'].astype(str)

In [ ]:
## TRANSFORMATION DATAFRAME
# concatenation dataframe
#df_btc_01 = pd.concat([df_btc_0, df_btc_1],axis=1)
#df_btc_01

In [ ]:
## VISUALISATION

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

#%matplotlib inline

#plt.figure(figsize=(15,6))

#x = df_btc["last_update"]
#y = df_btc["eur"]  
      
#plt.plot(x, 
 #        y,
 #        linewidth = 4)

#plt.title('Evolution du prix du Bitcoin en €')
#plt.xlabel('Date')
#plt.xticks(rotation = 45)
#plt.ylabel('Prix du Bitcoin (en euros et en dollar')



In [ ]:
## VISUALISATION

#import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#import plotly.graph_objects as go
#import plotly.express as px
#from plotly.subplots import make_subplots

#x = df_btc["last_update"]
#y = df_btc["eur"]  

#y1 = df_btc["usd"]

#fig1 = px.line(df_btc, x=x, y=y1)
#fig1.update_layout(legend_title_text = "Evolution du prix du Bitcoin en $")
#fig1.update_xaxes(title_text="Last Update")
#fig1.update_yaxes(title_text="btc dollar")

#fig.show()
#fig1.show()

In [48]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=("BTC euro last 24h", "BTC dollar last 24h","Capitalisation boursière last 24h","volume d'échange eur 24h"))

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=1, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="euro", row=1, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="dollar", row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="eur_market_cap", row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="eur_24h_vol", row=2, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur"],
name="cours BTC euro"
), row=1, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["usd"],
name="cours BTC dollar"
), row=2, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur_market_cap"],
name="capitalisation boursière"
), row=3, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur_24h_vol"],
name="Volume de négociation sur 24 h"
), row=4, col=1)

fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(height=1200, width=1000, title_text="Evolution du cours du Bitcoin", hovermode = "x unified" )
fig.show()

In [ ]:
## LE VOLUME D'ECHANGE est une métrique importante dans le suivi des marchés financiers pour plusieurs raisons:
### LIQUIDITE : Un volume d'échange élevé est généralement associé à une meilleure liquidité du marché. Plus le volume est élevé, plus il est facile d'acheter ou de vendre un actif sans causer de fluctuations de prix excessives.
### TENDANCE : Le volume d'échange peut fournir des informations sur la direction d'une tendance. Des volumes élevés pendant une hausse ou une baisse de prix peuvent indiquer la force de cette tendance.
### CONFIRMATION : Les analystes techniques utilisent souvent le volume d'échange pour confirmer ou invalider les signaux de trading. Par exemple, une hausse des prix avec un volume élevé peut indiquer une tendance haussière plus solide.
### VOLATILITE : Les périodes de faible volume d'échange peuvent contribuer à une plus grande volatilité des prix, car il peut être plus facile pour un petit nombre de transactions d'influencer les prix.
### EVALUATION DU MARCHE: Le volume d'échange peut également donner une idée de l'intérêt actuel des investisseurs pour un actif particulier. Un volume élevé peut indiquer un intérêt accru, tandis qu'un volume faible peut indiquer un manque d'intérêt ou de participation.
